# op-cat

## what?

<https://github.com/bip420/bip420>

op-cat能干啥，直接上 code

        case OP_CAT:
        {
            if (stack.size() < 2)
                return set_error(serror, SCRIPT_ERR_INVALID_STACK_OPERATION);
            valtype& vch1 = stacktop(-2);
            valtype& vch2 = stacktop(-1);
            if (vch1.size() + vch2.size() > MAX_SCRIPT_ELEMENT_SIZE) // MAX_SCRIPT_ELEMENT_SIZE=520
                return set_error(serror, SCRIPT_ERR_PUSH_SIZE);
            vch1.insert(vch1.end(), vch2.begin(), vch2.end());
            stack.pop_back();
        }
        break;


功能直接了当，将stack最上面的两个元素当作字符串直接拼接起来。

## history

之前中本聪是这样实现的，基本差不多

        case OP_CAT:
        {
            // (x1 x2 -- out)
            if (stack.size() < 2)
                return false;
            valtype& vch1 = stacktop(-2);
            valtype& vch2 = stacktop(-1);
            vch1.insert(vch1.end(), vch2.begin(), vch2.end());
            stack.pop_back();
            if (stacktop(-1).size() > 5000)
                return false;
        }
        break;

不过之前的实现被删除了，为啥呢。据说是之前有个关于 **OP_LSHIFT** 的bug，会导致bitcoin节点崩溃，然后解决这个bug的时候，就把 OP_CAT 也删除了。这个bug是什么鬼？

<https://www.reddit.com/r/btc/comments/9z2shd/some_thoughts_about_op_lshiftop_rshift/>

        case OP_LSHIFT:
            if (bn2 < bnZero)
                return false;
            bn = bn1 << bn2.getulong();
            break;
            
        case OP_RSHIFT:
            if (bn2 < bnZero)
                return false;
            bn = bn1 >> bn2.getulong();
            break;

貌似是因为 **bn2** 如果很大的话，比如10个亿，会让 **bn** 超级大，然后估计就 **OOM** 了，从而产生了 **DDOS** 攻击，但因为这个就把 **OP_CAT** 干掉了？神奇。可能是中本聪被搞怕了。

具体干掉这些 op code 的commit在这里

<https://github.com/bitcoin/bitcoin/commit/4bd188c4383d6e614e18f79dc337fbabe8464c82>

        if (opcode == OP_CAT ||
            opcode == OP_SUBSTR ||
            opcode == OP_LEFT ||
            opcode == OP_RIGHT ||
            opcode == OP_INVERT ||
            opcode == OP_AND ||
            opcode == OP_OR ||
            opcode == OP_XOR ||
            opcode == OP_2MUL ||
            opcode == OP_2DIV ||
            opcode == OP_MUL ||
            opcode == OP_DIV ||
            opcode == OP_MOD ||
            opcode == OP_LSHIFT ||
            opcode == OP_RSHIFT)
            return false;


## 用处

这个 OP_CAT 搞出来有啥用呢？

### 1. an equivocation bond （翻译为 模棱两可的债券 ？？）

<https://bitcoinops.org/en/topics/op_cat/>

    The primary expected use for OP_CAT is for data provided by the creator of a script to be concatenated with data provided by someone spending from that script.

gpt翻译为：

    OP_CAT 的主要预期用途是将脚本创建者提供的数据与使用该脚本的人提供的数据进行连接。

也就是将 pay 的时候的数据，和 spend 的时候的数据拼接起来。

这里用所谓的 **equivocation bond** 来举例。 **equivocation bond** 是什么？

gpt说：

    在区块链和分布式系统中，"模棱两可债券"是一种安全机制，用来防止参与者进行不诚实的行为，特别是发出矛盾的声明或行为，例如双重支付或发表冲突的声明。这通常涉及将一定数量的加密货币或代币存入一个债券中，如果证明参与者进行了模棱两可的行为，则这些资金可能会被没收。这种机制通过损失抵押金的风险来激励参与者保持诚实。

简单来说，就是你要是作出某些类型的瞎搞行为，你会受到惩罚，比如双花。

#### 先简单说一下 **schnorr签名**

<https://en.bitcoin.it/wiki/Schnorr>

签名过程如下（删除了很多细节），可以不用看细节，核心是最后得到的签名的样子

1. 已知: 私钥 **x** ，要签名的内容 **h**
2. 先搞出公钥: **P = x * G**
3. 先一个隐藏的随机数nonce: **k**
4. 搞出公开的nonce: **R = k * G**
5. **e = hash(h || R || P)**
6. **s = k + e * x**
7. 得到签名: **(R, s)**

验证签名的算法是

1. 已知 **(R, s)**, **h**
2. **s * G = (k + e * x) * G = k * G + e * x * G = R + e * P = R + hash(h || R || P) * G**

正常情况下，我们验证签名的byte codes如下

        <签名> <pubkey> CHECK_SIG

用户在支付的时候需要提供完整的签名

有了**OP_CAT** 之后，我们可以提供这样一个脚本供别人支付

        <nonce=R> OP_CAT <pubkey> CHECK_SIG

可以看出，用户只需要提供 **s** 就可以完成支付，这样有什么好处呢？

假设用户想要双花，因为 **e** 的hash中包含了 **h**，而双花的话 **h** 肯定是不一样的，所以 **e** 也就不一样了，所以 **s** 也就不一样了。这样我们就得到了两个不同的签名 **(s1, R)** 和 **(s2, R)** , 连个签名对应同一个私钥 **x** 。而且 **s1 - s2 = (k + e1 * x) - (k + e2 * x) = (e1 - e2) * x**，所以有 **x = (s1-s2)/(e1-e2)**。

所以完蛋了， **x** 算出来了，如果这个双花的家伙还有使用 **x** 为私钥的资产的话，就暴露了。

#### more

没有 OP_CAT 能完成这个任务吗？

这个例子是不是人为性太强了？我要瞎搞的话，是不是私钥就只干这个事情，不留别的资产了？

### 2. 高级契约 （写到一半发现，这个文章说到的高级契约，在OP_CAT已经支持的情况下也是实现不了的。。。 悲剧）

<https://www.wpsoftware.net/andrew/blog/cat-and-schnorr-tricks-i.html>

什么是契约？ btc本身就有一些机制

1. 最普通的 P2PK/P2PKH 之类的，我这笔钱，只能拥有某个地址的私钥的人才能拿走，别人休想
2. P2SH ， 只有知道指定脚本内容的人才能拿走
3. P2ANYONE，任何人都可以拿走
...

但是这些不够高级，更高级的体现在 btc 签名算法中所谓的 **hashtype** 

<https://en.bitcoin.it/wiki/OP_CHECKSIG>

我在签名一个交易的时候，我只签名一部分，而让别的部分可以变化。类似合同，我留下了一些空白页，允许别人填写内容。但是，和空白页不同的是，我不能允许别人瞎写，得符合一定之规

对于币的去向:

1. SIGHASH_ALL: 必须按照我指定的去向，btc去了哪里都被安排得明明白白的
2. SIGHASH_NONE: 我不关心去向。。。
3. SIGHASH_SINGLE: 别的我不管，但是我指定的去向必须满足，类似别人收多少钱我不管，但我得收这么多钱

对于币的来源:

1. SIGHASH_ANYONECANPAY: 别的人可以添加币的来源

可惜的是，目前来说，btc的所谓高级契约类型非常有限。对于 eth，因为它是图灵完备的，理论上，通过写合约的方式，可以搞出无数种千奇百怪的契约（或者叫智能合约）来

#### 一种特殊的契约

完美的契约起码包括，我可以通过程序限制一个tx的方方面面，比如要求 **给某某人至少多少钱，给另外一个人在满足什么的条件下至少多少钱** 之类的，但是这对btc确实太难了。

一种粗暴的办法是，我们限制花费我们的这个utxo的交易的txid比如是某个指定的值，这样其实就是限定了能够花费这笔钱的交易的所有信息，比如其他输入是什么，所有输出长什么样子。

一个交易的txid是怎么算出来的呢，是所有 input+output 然后hash，input主要是所花费的 utxo 的 txid和output index。所以交易的 txid 依赖于 utxo 的 txid。如果我们把 txid 写入到 utxo 的 script 中去，就会影响 utxo 的 txid，反过来就会影响当前交易的txid，这就循环依赖了。

**就是这里，这个绕不过去了**，所以上面描述的粗暴办法无法实现，但是文章得到一个有意思的东西，就是

#### 怎么在脚本中得到当前交易的txid？

考虑这个脚本 (pubkeyScript)

    <G> 2DUP SWAP CAT SWAP CHECKSIG

支付的时候需要提供 **s**，就是 schnorr 签名 (s, R) 中的 s，一次考虑stack上的情况

1. s
2. s G
3. s G s G
4. s G G s
5. s G G##s
6. s G##s G

然后是 CHECKSIG，G被当作pubkey，G##s 被当作签名。CHECKSIG要能过，就相当于要求 私钥是 **x=1**，因为 **G = 1 * G**，而秘密nonce **r=1**，也就是 **R = 1 * G**，这样的话，s 是什么？ **s = 1 + e**，其中 **e = hash(h || R || P) = hash(h || G || G)**。 e是可以提前算出来的，因此s也是。

好了，我们有一个基本上是当前tx的hash的东西 **s = 1 + e** 留在了栈上，我们可以对其进行要求了，比如满足什么条件我们才允许支付。。。

如前所述，最理想的是我们要求 **s** 是一个具体的值，这样就完全限制住了交易，但是这不可能，因为有循环依赖的问题。所以原文到这里就只能 **戛然而止**。

不过我们貌似还是可以做一些事情的。

比如我们要求 **s % 100000000 == 0**，这有点像 **POW**，要求对方必须花费很大力气才能穷举出一个交易。但是，好像 **并没有什么用处** 。